# Clustering for syndromic surveillance

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sb

# Import and prepare data

In [2]:
features = pd.read_pickle("data/rwanda/features.pickle")
diagnoses = pd.read_pickle("data/rwanda/diagnoses.pickle")
villages = pd.read_pickle("data/rwanda/rw-village-coordinates.pickle")

In [4]:
villages

,District,Sector,Cell,Village,coordinates,gps_coordinates,center_gps,latitude_village,longitude_village
0,nyarugenge,gitega,akabahizi,gihanga,"[(505804.1074000001, 4784697.3946), (505808.87...","[(-1.9473174395361728, 30.052174320526692), (-...","(-1.9481017833761631, 30.05229680936519)",-1.948102,30.052297
1,nyarugenge,gitega,akabahizi,iterambere,"[(505725.3504999997, 4785070.964), (505750.761...","[(-1.9439387164006545, 30.051466256263986), (-...","(-1.9456671128296834, 30.052606152215212)",-1.945667,30.052606
2,nyarugenge,gitega,akabahizi,izuba,"[(505689.33939999994, 4785047.2246), (505712.2...","[(-1.9441534370853488, 30.051142552237927), (-...","(-1.9447035908774273, 30.050792203288733)",-1.944704,30.050792
3,nyarugenge,gitega,akabahizi,nyaburanga,"[(506082.3940000003, 4784751.6763), (506066.27...","[(-1.9468264094804004, 30.054675879698678), (-...","(-1.947536650856193, 30.053989256753532)",-1.947537,30.053989
4,nyarugenge,gitega,akabahizi,nyenyeri,"[(505779.44519999996, 4784753.8181), (505787.7...","[(-1.9468071246056908, 30.051952611351428), (-...","(-1.9473031147058788, 30.051666486846646)",-1.947303,30.051666
...,...,...,...,...,...,...,...,...,...
14810,bugesera,shyara,rutare,gaseke,"[(500470.19570000004, 4749405.5013999995), (50...","[(-2.266514935034641, 30.004227547447556), (-2...","(-2.2698722752970775, 30.00040273164282)",-2.269872,30.000403
14811,bugesera,shyara,rutare,kamweru,"[(499906.30709999986, 4747938.3901), (499914.7...","[(-2.2797841651048216, 29.999157595874365), (-...","(-2.2836209316375453, 30.00400133954392)",-2.283621,30.004001
14812,bugesera,shyara,rutare,ruhanga,"[(498708.4414999997, 4748230.242699999), (4987...","[(-2.2771444704109625, 29.988387463507063), (-...","(-2.2815142038693947, 29.98870886971765)",-2.281514,29.988709
14813,bugesera,shyara,rutare,rutare,"[(500611.2845999999, 4748742.6085), (500635.35...","[(-2.272510436926198, 30.00549610575808), (-2....","(-2.277080507620433, 30.006476106560523)",-2.277081,30.006476


In [ ]:
complaints_older_kids = [
    "CC21 - General - 8341",
    "CC32 - Fever - 8081",
    "CC13 - Respiratory (Cough or difficult breathing) - 7808",
    "CC17 - Gastrointestinal (diarrhea, vomiting, feeding) - 7875",
    "CC10 - Skin / hair - 8346",
    "CC11 - Eye - 7803",
    "CC12 - Ear/Throat/Mouth - 8342",
    "CC18 - Accident /Muskulo-skeletal (incl. burns, wounds, poison) - 8343",
    "CC15 - Genitourinary (UTI, STI) - 8371",
    "CC16 - Neuro (Headache, stiff neck, neck pain) - 8372",
    "CC35 - Other - 8120"]

In [ ]:
complaints_infants = [
    "CC22 - General Assessment - 8352",
    "CC33 - Fever or convulsions or lethargy - 7391",
    "CC24 - Respiratory - 8370",
    "CC25 - Diarrhea, abdominal, gastro-intestinal - 8076",
    "CC27 - Eye complaint - 8066",
    "CC28 - Ear or mouth complaint - 8072",
    "CC23 - Yellow appearing skin or eyes (jaundice) - 7351",
    "CC30 - Malformation of birth anomaly - 7389",
    "CC29 - Feeding problem or weight concern - 8347",
    "CC31 - Injuries (birth and non-birth related) - 7977",
    "CC37 - Follow-up visit for a previous problem - 7521",
    "CC36 - Other - 8122"
]

# Perform clustering
Clustering approach selected based on the findings in `clustering-for-cdss-evaluation.ipynb`.

# Evaluation
... through ...
- diagnoses distribution
- accuracy and shap values of lgbm classifier

# Syndromic surveillance

## Demographic analysis of clusters

## Temporal analysis of clusters

## Spatial analysis of clusters

## Spatio-temporal analysis

## Outbreak detection